In [9]:
import importlib.machinery
import importlib.util
from pathlib import Path
import xml
import typing as T
import xml.etree.ElementTree as ET
from datetime import datetime
import requests_cache

HERE = Path(".").resolve()

In [4]:
MODULE_NAME = 'pelicanconf'
loader = importlib.machinery.SourceFileLoader(MODULE_NAME, str(HERE.parent / "pelicanconf.py") )
spec = importlib.util.spec_from_loader(MODULE_NAME, loader )
pelicanconf = importlib.util.module_from_spec( spec )
loader.exec_module(pelicanconf)

Theme is /home/jordan/projects/dgt/dgtstatic/custom-themes/putty2


In [5]:
pelicanconf.FEEDS

{'DGT_MEDIUM': 'https://medium.com/feed/@damngoodtech'}

In [6]:
feed = list(pelicanconf.FEEDS.values())[0]

In [11]:
session = requests_cache.CachedSession("dgtfeed")
resp = session.get(feed)

In [37]:
def textif(tree : ET, xpath : T.AnyStr):
    el = tree.find(xpath)
    if el is not None:
        return el.text

def parse_item_date(text : T.AnyStr, dtformat="%a, %d %b %Y %H:%M:%S %Z"):
    return datetime.strptime(text, dtformat)

class Item:
    
    def __init__(self, title, description, link, categories):
        self.title = title
        self.link = link
        self.categories = categories
        self.description = description
    
    def __str__(self):
        return "<Item title: '%s', link: '%s', categories: %s>" % (self.title, self.link, self.categories)
    
    @classmethod
    def from_element(cls, node : ET.Element):
        title = textif(node, "title")
        description = textif(node, "description")
        link = textif(node, "link")
        categories = [c.text for c in node.findall(".//category")]
        pubdate = textif(node, "pubdate")
        pubdate = parse_item_date(pudate) if pubdate else None
        
        return cls(title, description, link, categories)

class Feed:

    def __init__(self, title : T.AnyStr, description : T.AnyStr, link : T.AnyStr, items : T.Iterable[Item]):
        self.title = title
        self.description = description
        self.link = link
        self.items = items
    
    def __str__(self):
        return "<Feed title: '%s', description: '%s', link: '%s', items: [%s items]>" % (self.title, self.description, self.link, len(self.items))

    @classmethod
    def from_etree(cls, tree: ET):
        title = textif(tree, ".//title")
        desc = textif(tree, ".//description")
        link = textif(tree, ".//link")
        items = [Item.from_element(xitem) for xitem in tree.findall(".//item")]
        return cls(title, desc, link, items)

et = ET.fromstring(resp.text)

feed = Feed.from_etree(et)

In [38]:
[str(i) for i in feed.items]

["<Item title: 'ration The Unseen Deployment Puzzle: Why Heroku Won’t Deploy When Config Variables Are Involved', link: 'https://medium.com/@damngoodtech/ration-the-unseen-deployment-puzzle-why-heroku-wont-deploy-when-config-variables-are-involved-8da548cd5045?source=rss-c03d69f9e146------2', categories: ['heroku', 'web', 'technology', 'programming', 'devops']>",
 "<Item title: 'AI Weather Prediction Pipeline: From User Message to Accurate Forecast', link: 'https://medium.com/@damngoodtech/ai-weather-prediction-pipeline-from-user-message-to-accurate-forecast-abd7d861e9e0?source=rss-c03d69f9e146------2', categories: ['transformers', 'python', 'ai', 'programming', 'technolgoy']>",
 "<Item title: 'How to be a Django Form Mixologist', link: 'https://medium.com/@damngoodtech/how-to-be-a-django-form-mixologist-5d70b6beba37?source=rss-c03d69f9e146------2', categories: ['web-development', 'programming', 'technology', 'python', 'coding']>",
 "<Item title: 'How to Use AI Intelligently in Your Or